In [1]:
!python3 -m pip install -e ../.

Looking in indexes: https://pypi.org/simple, https://pypi.org/simple
Obtaining file:///Users/nelenj/Documents/projects/helmet
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for helmet (pyproject.toml) ... done
  Created wheel for helmet: filename=helmet-0.1.2-py3-none-any.whl size=1745 sha256=5e1e92d5fe06518cb5b9d96e61e0c65d7a11e9b2a1719b1e9ec8338fbbe3bbb0
  Stored in directory: /private/var/folders/f1/st6fs1q547zbfmzy6jr5b0km0000gp/T/pip-ephem-wheel-cache-62kl11_y/wheels/b5/be/d1/aaf642ac2f1062e27dc45a2559b93e86459c128f1f8ea5cea1
Successfully built helmet
  Attempting uninstall: helmet
    Found existing installation: helmet 0.1.2
    Uninstalling helmet-0.1.2:
      Successfully uninstalled helmet-0.1.2


In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# This to make sure that the changes in the module are reloaded
%reload_ext autoreload

In [13]:
# Platform configs
url = "https://api.jeroennelen.nl"
project_id = "66277759e1eacb39208e10ed"

project_setup = {
    "platform_url": url,
    "project_id": project_id
}

In [14]:
# checkpoint = "mlabonne/Gemmalpaca-2B"
# "meta-llama/Meta-Llama-3-8B"
device = "cpu"
# checkpoint = "bigscience/bloom-1b7"
checkpoint = "bigscience/bloom-560m"

model_setup = {
    "checkpoint": checkpoint,
    "model_type": "dec",
    "embeddings": "transformer.word_embeddings"
}

run_config = {
    "device": device,
    "max_new_tokens": 100,
    # "temperature": 1, # Not
    # e that if you add this, then you need to add "do_sample": True
    # "do_sample": True
}

In [15]:
import helmet

In [16]:
model = helmet.from_pretrained(project_setup=project_setup, model_setup=model_setup, run_config=run_config)

updates will be sent to https://api.jeroennelen.nl
setting up model with config {'checkpoint': 'bigscience/bloom-560m', 'model_type': 'dec', 'embeddings': 'transformer.word_embeddings'}
model loaded


In [ ]:
# !pip install huggingface_hub ipywidgets -q

In [ ]:
# This might be needed to load the model
# from huggingface_hub import login
# login()

In [8]:
from datasets import load_dataset

dataset = load_dataset("Helsinki-NLP/opus-100", name="en-it", split='test[:20]')

In [9]:
dataset[2]

{'translation': {'en': 'I, uh, I wrote you a letter when I was at the Academy.',
  'it': "E' un piacere conoscerla. Le... le ho scritto quando ero all'accademia."}}

In [ ]:
# prompt = dataset["validation"][1]
# # print(prompt)
# # prompt = prompt["question"]

In [21]:
source_lang = "en"
target_lang = "it"
prefix = "Translate English to Italian: "

def prompt_builder(sample):
    return prefix + sample['translation'][source_lang]


prompt = prompt_builder(dataset[2])
prompt

'Translate English to Italian: I, uh, I wrote you a letter when I was at the Academy.'

In [22]:
predict_config = {
    "max_new_tokens": 20,
    "generate_explanations": False,
}

output = model.predict(prompt, **predict_config)
output

updated app, result:  {'acknowledged': True, 'insertedId': '6627a1879f224fc8058a69c9'}


' I was in the class, and I was thinking about the future. I was thinking about the future'

In [24]:
for i in range(20):
    prompt = prompt_builder(dataset[i])
    result = model.predict(prompt, **predict_config)
    print(result)

updated app, result:  {'acknowledged': True, 'insertedId': '6627a2289f224fc8058a69ca'}
 He was a member of the Royal Navy and was a member of the Royal Air Force. He was
updated app, result:  {'acknowledged': True, 'insertedId': '6627a22b9f224fc8058a69cb'}
.", $lang->translate('translate', 'English to Italian: %s', $translate))
updated app, result:  {'acknowledged': True, 'insertedId': '6627a22e9f224fc8058a69cc'}
 I was in the class, and I was thinking about the future. I was thinking about the future
updated app, result:  {'acknowledged': True, 'insertedId': '6627a2309f224fc8058a69cd'}
 and kills the boss. The boss is a very powerful boss, and he has a very powerful weapon
updated app, result:  {'acknowledged': True, 'insertedId': '6627a2339f224fc8058a69ce'}
"
"So, what do you think?"
"I'm not sure."
"
updated app, result:  {'acknowledged': True, 'insertedId': '6627a2369f224fc8058a69cf'}
 He was a very good player. He was very good at playing poker. He was very good at
updated app, re

In [23]:
model.saliency_explainer("6627a1879f224fc8058a69c9")

/Users/nelenj/Documents/projects/helmet/src/helmet/explainers/gradients.py:67: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids_new = torch.tensor(input_ids.clone().detach())


finished token 20 of 40
finished token 21 of 40
finished token 22 of 40
finished token 23 of 40
finished token 24 of 40
finished token 25 of 40
finished token 26 of 40
finished token 27 of 40
finished token 28 of 40
finished token 29 of 40
finished token 30 of 40
finished token 31 of 40
finished token 32 of 40
finished token 33 of 40
finished token 34 of 40
finished token 35 of 40
finished token 36 of 40
finished token 37 of 40
finished token 38 of 40
finished token 39 of 40
updated app, result:  {'acknowledged': True, 'modifiedCount': 1, 'upsertedId': None, 'upsertedCount': 0, 'matchedCount': 1}


SaliencyExplanation(explanation_method='saliency', input_attributions=[array([-7.6048754e-02,  1.2228975e-01,  1.5276311e-03, -3.7846822e-01,
       -1.9072058e-02,  4.7364280e-02,  1.3100481e-01,  1.1092962e-02,
       -2.9582655e-02, -1.7736435e-02, -1.8269683e-03, -9.7456342e-03,
       -6.8274751e-02,  6.4783095e-04,  1.1832054e-02,  4.3987972e-03,
       -1.7653212e-04,  2.8490179e-04, -1.9979328e-02, -4.8645642e-02],
      dtype=float32), array([-0.06080727,  0.02301275, -0.00332828, -0.24189119,  0.03322918,
        0.02943031,  0.13989455,  0.01109391,  0.0033715 ,  0.0606625 ,
       -0.04196557, -0.01862948, -0.03973954,  0.00603922,  0.01853734,
        0.01498539,  0.02272374,  0.00752176,  0.01610308,  0.0149111 ,
       -0.19212225], dtype=float32), array([ 0.00676765,  0.03971576,  0.00534493, -0.19623528,  0.00887387,
       -0.02964568,  0.07064395,  0.00883238, -0.00387936,  0.03694752,
       -0.0127468 , -0.00405384,  0.02846467, -0.0100301 , -0.0270422 ,
        0.